# Temporary Tables 
A temporary table in SQL Server, as the name suggests, is a database table that exists temporarily on the database server. A temporary table stores a subset of data from a normal table for a certain period of time.



## Use cases
- One of the most often used scenarios for SQL Server temp tables is within a loop of some sort. For example, you want to process data for a SQL statement and it you need a place to store items for your loop to read through. It provides a quick and efficient means to do so. 
- Another reason to use SQL Server temp tables is you have some demanding processing to do in your sql statement. Let’s say that you create a join, and every time you need to pull records from that result set it has to process this join all over again. Why not just process this result set once and throw the records into a SQL temp table? Then you can have the rest of the sql statement refer to the SQL temp table name. Not only does this save on expensive query processing, but it may even make your code look a little cleaner.

## Global vs Local
### Local
- Stored in `tempdb` under `System Databases`
- Can only be used:
 - In the creating sessions
 - At the creating level or underlying levels
- Deleted when the creating level goes out-of-scope

#### Example Local Temp. Table

In [ ]:
DROP TABLE IF EXISTS #MyOrderTotalsByYear;

CREATE TABLE #MyOrderTotalsByYear
( 
 OrderYear INT NOT NULL PRIMARY KEY
,Quantity  INT NOT NULL 
);  

-- Populate the Local Temp. Table
INSERT INTO #MyOrderTotalsByYear
(
 OrderYear
,Quantity
)
SELECT 
 YEAR([Order].OrderDate) AS OrderYear
,SUM(Detail.Quantity) AS Quantity
FROM Orders AS [Order] 
	JOIN OrdersDetail AS Detail ON [Order].OrderId = Detail.OrderId
GROUP BY YEAR(OrderDate);  

-- Query the Local Temp. Table
SELECT 
 CurrentYear.OrderYear
,CurrentYear.Quantity AS 'Current Year Quantity'
,PreviousYear.Quantity AS 'Previous Year Quantity'
FROM #MyOrderTotalsByYear AS CurrentYear
	LEFT JOIN #MyOrderTotalsByYear AS PreviousYear ON CurrentYear.OrderYear = PreviousYear.OrderYear + 1;



### Global
- Stored in `tempdb` under `System Databases`
- Visible in **all** sessions
- Removed if creating session disconnects **and** there are no more references

#### Example Global Temp. Table

In [ ]:
CREATE TABLE ##MyGlobalTable 
(
 Id CHAR(4) NOT NULL PRIMARY KEY
,[Value] INT NOT NULL );

INSERT INTO ##MyGlobalTable(Id, [Value]) 
VALUES('ABCD',10);

SELECT [Value] 
FROM ##MyGlobalTable 
WHERE Id = 'ABCD';

-- DROP TABLE ##MyGlobalTable;


## Use case
- Due to the Brexit we want to delete all orders that are not yet shipped and that contain products that are supplied by a supplier from the UK.
- We first have to delete the ordersdetail because of the FK constraint with orders
- But after deleting the ordersdetail we loose the link with the supplier and don’t kwow which orders to delete anymore
- Solution: save the orderid’s in a `temporary` table. 



In [ ]:
CREATE PROCEDURE DeleteOrdersUK 
 @deletedorders int output
AS
BEGIN
SET NOCOUNT ON
CREATE TABLE #OrdersUK 
(
 orderid int
)

INSERT INTO #OrdersUK
SELECT DISTINCT od.orderid 
FROM ordersdetail od
    JOIN orders o on od.orderid = o.orderid 
WHERE productid IN 
(SELECT productid 
 FROM product p 
    JOIN supplier s on p.SupplierID=s.SupplierID 
 WHERE country='UK'
) AND o.Shipped=0;

DELETE FROM  ordersdetail WHERE orderid in (SELECT orderid FROM #OrdersUK)
DELETE FROM orders	WHERE orderid in (SELECT orderid FROM #OrdersUK)
SET @deletedorders = @@ROWCOUNT
END


### Executing the procedure

In [ ]:
-- test of procedure DeleteOrdersUK
begin transaction

select od.orderid from ordersdetail od join orders o on od.orderid=o.orderid 
where productid in 
(select productid from product p join supplier s on p.SupplierID=s.SupplierID where country='UK') and o.Shipped=0;

declare @nroforders int
exec DeleteOrdersUK @nroforders out
print 'Nr of deletedorders = ' + cast(@nroforders as varchar)

select od.orderid from ordersdetail od join orders o on od.orderid=o.orderid 
where productid in 
(select productid from product p join supplier s on p.SupplierID=s.SupplierID where country='UK') and o.Shipped=0;

rollback
